# 第8章 ディープラーニングを用いた因果探索

#### ・本章ではディープラーニングを用いた因果探索について解説、実装する .

## 8-1 因果探索とGAN(Generative Adversarial Networks)の関係

#### ・2020年において、因果推論の分野でもディープラーニングを利用した研究が進んでいる .　具体例として、

#### グラフニューラルネットワークを用いた因果探索
#### 深層強化学習を用いた因果探索
#### GAN(Generative Adversarial Networks)を用いた因果探索

#### などが挙げられる .

#### ・本章ではディープラーニングを用いた因果探索手法の中でも、"SAM(Structural Agnostic Model)"について解説、実装する .　SAMはGANを用いた因果探索手法である .

#### ・本章の内容はPyTorchを用いたディープラーニングとGANの実装経験がないと理解が難しい部分が多いらしいです .

#### ・本章で出てくる実装はSAM論文の著者らのGitHubのコードを一部参考、使用している .

![alt text](pict1.png)

## GAN(Generative Adversarial Networks)とは

#### ・GANは大量の画像データを学習に使用し、実際には存在していない架空の画像を生成する技術として有名である .

#### GANは生成器(Generator,以下G)と呼ばれるニューラルネットワークと、識別器(Discriminator,以下D)と呼ばれる2種類のニューラルネットワークから構成される .

#### ・画像を生成する際に使用するのは、「学習済みの生成器G」のみである .　学習済みの生成器Gに入力としてノイズを与えると、そのノイズの値に応じた架空の画像データが生成され、Gから出力される .

#### 例えば、入力ノイズの次元が20次元、出力される画像のサイズが縦横30ピクセルずつであれば、テンソルサイズ[20]のノイズ入力を入力として、テンソルサイズ[3,30,30]の出力を作り出す .　出力テンソルの最初の次元の3はRGBの各色チャンネルを示す .

#### ・学習済み生成器Gを作る際に、手書き数字の画像を学習データとして与えた場合、手書き数字の画像を出力するような生成器Gを構築できる .

#### また、GANの種類によっては入力にノイズだけでなく、条件(手書き数字画像であれば0や1など数字の種類を指定)を入力する場合がある .　この場合は条件に応じた任意の数字の画像を生成させることができる(Conditional GANと呼ぶ .) .

### ・ここで重要な点は、"生成器Gはノイズを入力に人が数字と認識できる画像を生成してくれる"点である .　例えば、3×30×30の画像サイズにおいて、各値は0から255の256通りの値をとるため、生成できる画像パターンは膨大になる .　この膨大なパターンの中から、人が見たときに手書き数字画像と思えるパターンを出力してくれるのが生成器Gである .

#### このとき、生成される画像は基本的には学習に使用した画像には含まれていないパターンの画像となる .　学習に使用した画像を表示するのではなく、学習に使用した画像の特徴をもった、新たな画像を生成してくれる .

#### ・では、そのような生成器Gを構築するために生成器Gのニューラルネットワークをどのように学習させれば良いかが問題となる .　ここで識別器Dが登場する .

#### 生成器Gが生成した画像が数字に見えるかどうかを、いちいち人が判定することは大変で時間的にも困難なので、識別器Dが人に代わって、「生成器Gが生成した画像が数字に見えるかどうか」を判定する .　正確には、「数字に見えるかどうか」の判定は難しいので、「学習データセットにある画像か、それともGで生成された画像か？」を判定させる .

### 識別器Dが学習データセットにある画像かGで生成された画像かうまく区別が付かないようになれば、Gで生成される画像は学習データの特徴をもった、人が見ても手書き数字に見える画像だと判断できる .

#### ・この学習の際に、識別器Dは学習データセットの画像と生成画像の判定がうまくできない状態からスタートし、生成器Gもまるで砂嵐のような画像しか生成できない状態からスタートする .

#### そして、生成器Gは「識別器Dが学習データセットの画像と勘違いする画像を生成するように」、識別器Dは「学習データセットにある画像か、それとも生成された画像か判定できるように」、それぞれのニューラルネットワークを更新していく .

#### 識別器Dも初期状態からスタートする理由は、はじめから完璧にGの生成画像を見抜かれてしまうと、Gの学習がうまくいかないからである .　DとGを徐々に切磋琢磨させながら、学習を進める .

#### ・これらを踏まえると、最終的に学習済み生成器Gは人が見たときに、学習データセットにある画像のような画像を生成できるようになる .　これがGANの大まかなイメージである .

#### ・本章の因果探索の場合では、「観測したデータと同じような特徴のデータを生み出す生成器Gを学習させることができれば、この生成器Gから観測データが生まれるしくみを解き明かし、因果ダイアグラムを描くことができる .　これが因果探索にGANを用いるモチベーションとなる .

## 8-2 SAM(Structural Agnostic Model)の概要

#### ・本節では、GANを用いた因果探索手法であるSAM(Structural Agnostic Model)の、生成器Gおよび識別器Dの概要を、図を用いて解説する .

## 識別器Dのネットワーク構造

#### ・初めに識別器Dのネットワーク構造について解説する .

#### ・識別器Dへの入力テンソルサイズは[mini_batch数,観測変数の数]である .　この入力テンソルの要素の値は観測データの変数の値であり、事前に標準化しておく .

#### ・識別器Dの出力テンソルはサイズが[mini_batch数,1]である .　出力テンソルのサイズ1は、「入力データが学習データセットに含まれているものか(すなわち実際に観測したものか)、それとも生成器Gが生成したものか」を判定する .

#### このテンソルの値は、マイナスの値であれば、偽物の生成器が生成したデータと判定し、プラスの値であれば学習データセットに含まれていた観測データを意味する .　値の絶対値が大きければ大きいほど、その確信度が高くなる .

![alt text](pict2.png)

#### ・図8.2.1では入力データのミニバッチ数を2,000、観測データの変数の種類を6と仮定している .　入力データははじめに"$L$" と書かれた「線形全結合層(fully-connected Linear Layer)」に入る .

#### ここでは出力次元数は変数dnhで決められ、図8.2.1ではdnh=200としている .

#### ・その後、データに1次元のバッチノーマライゼーションが実行される .　200次元になったミニバッチ2,000個のデータに対して、各200次元それぞれが平均0,標準偏差1になる変換を学習させる .

#### ・バッチノーマライゼーションのあと活性化関数のLeakyReLUで処理される .　一般的な活性化関数ReLUはマイナスの入力に対する出力が0になるが、LeakyReLUではマイナスの入力に対しても入力に応じた値(ここではPyTorchのデフォルトである0.2×入力値)が出力される .

#### ・活性化関数LeakyReLUを通ったデータのテンソルサイズは[2000,200]となっている .　その後、もう一度、線形全結合層、バッチノーマライゼーション、LeakyReLUに通す .

#### ・最後に線形全結合層(入力は200変数、出力は1変数)に通す .　この線形全結合層から出てくるテンソルが識別器Dの判定結果となり、テンソルサイズは[2000,1]となる .

## 生成器Gの概要

#### ・識別器Dは一般的なGANで使用される構成とさほど変わらず、画像生成GANの識別器Dでは2次元(縦、横)のバッチノーマライゼーションであるところが、1次元バッチノーマライゼーションに代わっただけである .

#### ・一方で生成器Gは、因果探索したいデータの生成過程を担い、変数間の因果ダイアグラムのつながりを求められる必要があるため、因果探索に対応した独特な形になる .

#### ここで、SAMの生成器Gがどのようにデータを生成するのか、そして識別器Dがどのように生成データを判定するのかが重要である .

#### ・例えば観測変数が6種類あったとする .

### この場合SAMの生成器Gは、「6種類の変数の値6つを同時に1回で生成するのではなく、とある変数1つに着目し、残りの5つの変数には観測データを与え、入力ノイズに応じて、とある変数1つだけを生成する .

### そして、識別器Dは「1つの変数の値がGでの生成データで、残りの5つが観測データのfakeケースと、6つ全てが観測データのケース、入力されたデータはこのどちらなのか？を判定する .」

#### ・変数が6つある場合は、変数を変えながら上記の過程を6回実施する .

#### ・この内容を図解したのが図8.2.2になる .

![alt text](pict3.png)

#### ・この生成器での生成データを識別器に入力するときには、図8.2.3のように工夫して与える .　識別器には生成データと一緒に観測データも与え、観測データから1種類の変数の値だけを生成データに置き換えて、それが生成されたデータと見破れるかを試す .

#### そのため、生成データを識別器に与える場合は、変数の数だけ識別器の判定結果が出る .

![alt text](pict4.png)

## 生成器Gのネットワーク構造

#### ・データ生成では、「ノイズを入力してとある1変数の値のみを生成データとして作成、その他の変数の値には観測データを与える」、これを変数の種類数だけ実施する .　この特殊なデータ生成のルールに応じて、ネットワークが複雑な構造をしている .

#### ・生成器Gのポイントは、変数の種類数だけ生成過程を繰り返すが、実際に繰り返す時間がもったいないので、"変数の種類数だけ生成過程を繰り返す操作を行列演算として実装する" .

#### この操作を実行するために、PyTorchにはない独自のモジュールとして、Linear3DモジュールとChannelBatchNorm1dモジュールを使用する .

#### これらのモジュールはSAMオリジナルである .　これらのモジュールがどのような操作をしているのかを解説する .

#### ・図8.2.4にSAMの生成器Gのネットワーク構造を示す .　最初のLinear3Dモジュールは基本的には単なる全結合層である .　ただし、1変数を除く観測されたデータとノイズを入力に、線形和の計算を実装する .

#### ・その後バッチノーマライゼーションで出力を標準化し、平均0、分散1に近づくように変換する .　この際に、BatchNorm1dを実行したいが、独自のLinear3Dの出力が、2次元ではなく3次元のテンソルになっていて、PyTorchのBatchNorm1dが適用できない .

#### そこで、内部で一度2次元にして、BatchNorm1dを適用し、再度元のテンソルサイズに戻すバッチノーマライゼーション操作として、ChannelBatchNorm1dモジュールを用意し、適用する .

#### その後、再度Linear3Dによる線形変換を実施して、最終的に[minibatch数、変数の数]の生成データを出力する .　以上が、SAMでの生成器Gの大まかなネットワーク構造である .

![alt text](pict5.png)

#### ・次に、因果ダイアグラムの構造を生成器Gに取り込む方法について解説する .

#### ・上記の図8.2.4の生成器のネットワーク概要図では掲載を省略しているが、生成器にはネットワーク構造のマトリクス$M$(SAMの論文のなかではstructual gateと記載)と、1つ目のLinear3Dの複雑さをコントロールするマトリクス"Z"(論文中ではfunctional gateと記載)が存在している .

#### この2つのマトリクスは要素に0か1の値をとる .　例えば、観測変数が3種類で$M$が、[[0,1,1],[0,0,1],[0,0,0]]であった場合、変数1から変数2と変数3へ因果がつながっている .　さらに変数2から変数3へもつながっている .　変数3からはどこへもつながっていない、ということになる .

#### ・Linear3Dの複雑さをコントロールするマトリクス$Z$の場合は、Linear3Dの出力要素のうち、$Z$の要素が0に対応する値には0が掛け算されて、実質的には使用されないようになる(結果、生成過程の複雑さが減る) .

#### ・以上の概念を図8.2.4に加えると、図8.2.5となる .

![alt text](pict6.png)

#### ・ネットワーク構造を示すマトリクス$M$と、1つ目のLinear3Dの複雑さをコントロールするマトリクス$Z$は生成器Gのforward関数(順伝搬)の計算時に、生成器Gに与える .

#### ・生成器Gの学習時には、このネットワーク構造のマトリクス$M$と、複雑さマトリクス$Z$も学習させ、それぞれのマトリクスの各要素が0になるか1になるかを学習させる .　そして、このネットワーク構造のマトリクス$M$こそが、因果ダイアグラムの構造を示すマトリクスとなる .

## 因果構造マトリクス$M$と複雑さマトリクス$Z$について

#### ・この2つのマトリクスの学習について解説する .

#### ・変数間の因果のつながり、因果ダイアグラムの形を示す因果構造マトリクス$M$と、生成器Gの1つ目の全結合の複雑さをコントロールするマトリクス$Z$は、どちらもその要素に0もしくは1の値をもつ .

#### ・しかしながら、ディープラーニングにおいて、0か1のような離散的な値をとる要素の学習方法は一般的ではない .　例えば、分類問題でもディープラーニングのネットワークからは最終的に連続値が出力され、その出力に対してソフトマックス関数を用いてクラス間で正規化して、最も大きな値を推論したラベルとしていた .

#### ・このような離散値を学習させるためには、Gumbel-Softmaxと呼ばれる技術を利用する .　Gumbel-Softmaxを利用して0か1を要素にもつマトリクスを作るモジュールとして、MatrixSamplerを用意する .　今回、モジュールMatrixSamplerはLinear3D、ChannelBatchNorm1dと同じく、SAMオリジナルの実装モジュールを使用する . 

### ・本書では、「通常はディープラーニングでは連続値を出力して学習させるが、0、1のような離散値を出力できるモジュールもGumbel-Softmaxを利用すれば作ることができ、SAMではそれを使用している」程度に理解できていればよい .

## 8-3 SAMの識別器Dと生成器Gの実装

#### ・本章では、7.5節でも使用した、疑似データ「上司向け : 部下とのキャリア面談のポイント研修」を使用する .　データの因果構造は図8.3.1の通りである .　観測変数は6種類で、$(x,Z,Y,Y2,Y3,Y4)$である .

![alt text](pict7.png)

#### ・また、ネットワーク構造をマトリクスで表すと、

$$
M = \begin{pmatrix}
0 & 1 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0
\end{pmatrix}
$$


#### となる.

### プログラム前の実行

In [2]:
# PyTorchのバージョンを下げる
!pip install torch==1.4.0+cu92 torchvision==0.5.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.4.0+cu92 (from versions: 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1)

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for torch==1.4.0+cu92


In [3]:
import torch 
print(torch.__version__)  # 元は1.5.0+cu101、versionを1.4に下げた

2.7.1+cu126


In [4]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

In [5]:
# 使用するパッケージ（ライブラリと関数）を定義
# 標準正規分布の生成用
from numpy.random import *

# グラフの描画用
import matplotlib.pyplot as plt

# その他
import pandas as pd

# シグモイド関数をimport
from scipy.special import expit

### データの作成

In [6]:
# データ数
num_data = 2000

# 部下育成への熱心さ
x = np.random.uniform(low=-1, high=1, size=num_data)  # -1から1の一様乱数

# 上司が「上司向け：部下とのキャリア面談のポイント研修」に参加したかどうか
e_z = randn(num_data)  # ノイズの生成
z_prob = expit(-5.0*x+5*e_z)
Z = np.array([])

# 上司が「上司向け：部下とのキャリア面談のポイント研修」に参加したかどうか
for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)

# 介入効果の非線形性：部下育成の熱心さxの値に応じて段階的に変化
t = np.zeros(num_data)
for i in range(num_data):
    if x[i] < 0:
        t[i] = 0.5
    elif x[i] >= 0 and x[i] < 0.5:
        t[i] = 0.7
    elif x[i] >= 0.5:
        t[i] = 1.0

e_y = randn(num_data)
Y = 2.0 + t*Z + 0.3*x + 0.1*e_y 


# 本章からの追加データを生成

# Y2：部下当人のチームメンバへの満足度 1から5の5段階
Y2 = np.random.choice([1.0, 2.0, 3.0, 4.0, 5.0],
                      num_data, p=[0.1, 0.2, 0.3, 0.2, 0.2])

# Y3：部下当人の仕事への満足度
e_y3 = randn(num_data)
Y3 = 3*Y + Y2 + e_y3

# Y4：部下当人の仕事のパフォーマンス
e_y4 = randn(num_data)
Y4 = 3*Y3 + 2*e_y4 + 5

### データをまとめた表を作成する

In [7]:
df = pd.DataFrame({'x': x,
                   'Z': Z,
                   't': t,
                   'Y': Y,
                   'Y2': Y2,
                   'Y3': Y3,
                   'Y4': Y4,
                   })

del df["t"]  # 変数tは観測できないので削除

df.head()  # 先頭を表示

,x,Z,Y,Y2,Y3,Y4
0,-0.616961,1.0,2.286924,2.0,8.732544,30.326507
1,0.244218,1.0,2.864636,3.0,10.743959,37.149014
2,-0.124545,0.0,2.198515,3.0,10.569163,38.481185
3,0.570717,1.0,3.230572,3.0,12.312526,43.709229
4,0.559952,0.0,2.459267,5.0,12.418739,40.833938


## CausalDiscoveryToolboxのインストール

#### ・SAM論文の著者らが整備している、SAMを含んだ因果探索のPythonパッケージ「CausalDiscoveryToolbox」をインストールする .　本章ではこのCausalDiscoveryToolboxにあるSAMの一部モジュール(Linear3D、ChannelBatchNorm1d、MatrixSampler)を使用する .

In [8]:
!pip install cdt==0.5.18


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


## SAMの識別器Dの実装

#### ・識別器Dをクラス「SAMDiscriminator」として実装する .

#### ・実装するネットワーク構造は、全結合層、バッチノーマライゼーション、LeakyReLUを2回繰り返して、最後に全結合層で出力を得る(8.2節) .

#### ・SAMは、識別器Dも生成器Gもforward関数(順伝搬)が複雑である .　生成器Gで全観測変数を生成するのではなく、1変数のみを生成データ、他は観測データとするため、実装が複雑になる .

#### ・識別器Dのforward計算は、入力が観測データのときは単純で、ネットワークに入力して判定するだけである .　入力が生成データの場合は複雑になる .

#### SAMの識別器Dの実装は以下の通りである .

In [9]:
# PyTorchから使用するものをimport
import torch
import torch.nn as nn


class SAMDiscriminator(nn.Module):
    """SAMのDiscriminatorのニューラルネットワーク
    """

    def __init__(self, nfeatures, dnh, hlayers):
        super(SAMDiscriminator, self).__init__()

        # ----------------------------------
        # ネットワークの用意
        # ----------------------------------
        self.nfeatures = nfeatures  # 入力変数の数

        layers = []
        layers.append(nn.Linear(nfeatures, dnh))
        layers.append(nn.BatchNorm1d(dnh))
        layers.append(nn.LeakyReLU(.2))

        for i in range(hlayers-1):
            layers.append(nn.Linear(dnh, dnh))
            layers.append(nn.BatchNorm1d(dnh))
            layers.append(nn.LeakyReLU(.2))

        layers.append(nn.Linear(dnh, 1))  # 最終出力

        self.layers = nn.Sequential(*layers)

        # ----------------------------------
        # maskの用意（対角成分のみ1で、他は0の行列）
        # ----------------------------------
        mask = torch.eye(nfeatures, nfeatures)  # 変数の数×変数の数の単位行列
        self.register_buffer("mask", mask.unsqueeze(0))  # 単位行列maskを保存しておく

        # 注意：register_bufferはmodelのパラメータではないが、その後forwardで使う変数を登録するPyTorchのメソッドです
        # self.変数名で、以降も使用可能になります
        # https://pytorch.org/docs/stable/nn.html?highlight=register_buffer#torch.nn.Module.register_buffer
def forward(self, input, obs_data=None):
        """順伝搬の計算
        Args:
            input (torch.Size([データ数, 観測変数の種類数])): 観測したデータ、もしくは生成されたデータ
            obs_data (torch.Size([データ数, 観測変数の種類数])):観測したデータ
        Returns:
            torch.Tensor: 観測したデータか、それとも生成されたデータかの判定結果
        """

        if obs_data is not None:
          # 生成データを識別器に入力する場合
            return [self.layers(i) for i in torch.unbind(obs_data.unsqueeze(1) * (1 - self.mask)
                                                         + input.unsqueeze(1) * self.mask, 1)]
            # 対角成分のみ生成したデータ、その他は観測データに
            # データを各変数ごとに、生成したもの、その他観測したもので混ぜて、1変数ずつ生成したものを放り込む
            # torch.unbind(x,1)はxの1次元目でテンソルをタプルに展開する
            # minibatch数が2000、観測データの変数が6種類の場合、
            # [2000,6]→[2000,6,6]→([2000,6], [2000,6], [2000,6], [2000,6], [2000,6], [2000,6])→([2000,1], [2000,1], [2000,1], [2000,1], [2000,1], [2000,1])
            # returnは[torch.Size([2000, 1]),torch.Size([2000, 1]),torch.Size([2000, 1], torch.Size([2000, 1]),torch.Size([2000, 1]),torch.Size([2000, 1])]

            # 注：生成した変数全種類を用いた判定はしない。
            # すなわち、生成した変数1種類と、元の観測データたちをまとめて1つにし、それが観測結果か、生成結果を判定させる

        else:
            # 観測データを識別器に入力する場合

            return self.layers(input)
            # returnは[torch.Size([2000, 1])]

def reset_parameters(self):
        """識別器Dの重みパラメータの初期化を実施"""
        for layer in self.layers:
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()

## 生成器Gの実装

#### ・続いてデータを生成する生成器Gを作成する .　この生成器Gに対して、観測データと同じ傾向を持つデータを生成する方法を学習させることで、データ生成のメカニズムを解き明かし、因果関係を明らかにする .

#### ・今回はSAMの著者らのパッケージから、3つのモジュール、Linear3D、ChannelBatchNorm1d、MatrixSamplerを利用する .　コード内にこれらのモジュールの実装へのリンクを記載する .

#### ・生成器Gの実装は以下の通りである .　Linear3Dの全結合層、バッチノーマライゼーション、活性化関数Tanh、そして再度Linear3Dの全結合層を通る(8.2節) .

#### ・変数skeletonは、因果ダイアグラムの構造を示すマトリクスの変数adj_matrixにかけ算して、自分から自分への因果(adj_matrixの対角成分)を0にするために作成、使用している .

In [10]:
from cdt.utils.torch import ChannelBatchNorm1d, MatrixSampler, Linear3D


class SAMGenerator(nn.Module):
    """SAMのGeneratorのニューラルネットワーク
    """

    def __init__(self, data_shape, nh):
        """初期化"""
        super(SAMGenerator, self).__init__()

        # ----------------------------------
        # 対角成分のみ0で、残りは1のmaskとなる変数skeletonを作成
        # ※最後の行は、全部1です
        # ----------------------------------
        nb_vars = data_shape[1]  # 変数の数
        skeleton = 1 - torch.eye(nb_vars + 1, nb_vars)

        self.register_buffer('skeleton', skeleton)

        # 注意：register_bufferはmodelのパラメータではないが、その後forwardで使う変数を登録するPyTorchのメソッドです
        # self.変数名で、以降も使用可能になります
        # https://pytorch.org/docs/stable/nn.html?highlight=register_buffer#torch.nn.Module.register_buffer

        # ----------------------------------
        # ネットワークの用意
        # ----------------------------------
        # 入力層（SAMの形での全結合層）　
        self.input_layer = Linear3D(
            (nb_vars, nb_vars + 1, nh))  # nhは中間層のニューロン数
        # https://github.com/FenTechSolutions/CausalDiscoveryToolbox/blob/32200779ab9b63762be3a24a2147cff09ba2bb72/cdt/utils/torch.py#L289

        # 中間層
        layers = []
        # 2次元を1次元に変換してバッチノーマライゼーションするモジュール
        layers.append(ChannelBatchNorm1d(nb_vars, nh))
        layers.append(nn.Tanh())
        self.layers = nn.Sequential(*layers)

        # ChannelBatchNorm1d
        # https://github.com/FenTechSolutions/CausalDiscoveryToolbox/blob/32200779ab9b63762be3a24a2147cff09ba2bb72/cdt/utils/torch.py#L130

        # 出力層（再度、SAMの形での全結合層）
        self.output_layer = Linear3D((nb_vars, nh, 1))
            
def forward(self, data, noise, adj_matrix, drawn_neurons=None):
        """順伝搬の計算
        Args:
            data (torch.Tensor): 観測データ
            noise (torch.Tensor): データ生成用のノイズ
            adj_matrix (torch.Tensor): 因果関係を示す因果構造マトリクスM
            drawn_neurons (torch.Tensor): Linear3Dの複雑さを制御する複雑さマトリクスZ
        Returns:
            torch.Tensor: 生成されたデータ
        """

        # 入力層
        x = self.input_layer(data, noise, adj_matrix *
                             self.skeleton)  # Linear3D

        # 中間層（バッチノーマライゼーションとTanh）
        x = self.layers(x)

        # 出力層
        output = self.output_layer(
            x, noise=None, adj_matrix=drawn_neurons)  # Linear3D

        return output.squeeze(2)

def reset_parameters(self):
        """重みパラメータの初期化を実施"""

        self.input_layer.reset_parameters()
        self.output_layer.reset_parameters()

        for layer in self.layers:
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()

No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.


## 因果構造マトリクス$M$と複雑さマトリクス$Z$の実装

#### ・変数間の因果の関係性を示す因果構造マトリクス$M$と、生成器Gの1つ目の全結合の複雑さをコントロールするマトリクス$Z$は、どちらもその要素に0もしくは1の値をもつ .　前節で解説したようにこのような離散値を実現するためにGumbel-Softmaxを利用したモジュールを作成する .

#### ・本書ではこのモジュールはMatrixSamplerとして、SAMの著者らのモジュールを使用するので、この部分は実装しないとする .

#### ・因果構造マトリクス$M$と複雑さマトリクス$Z$はSAMの生成器のforward関数の引数で使用されており、それぞれadj_matrixとdrawn_neuronsとしている(厳密には、adj_matrixは因果構造マトリクス$M$にノイズの項が加わったものである) .

## 8-4 SAMの損失関数の解説と因果探索の実装

#### ・本章ではSAMの損失関数の解説と実装を行う .　また、SAMの学習を実施する部分も合わせて実装する .

## DAGを生み出す損失関数 : NO TEARS

#### ・因果探索を行うにあたり、変数間の因果関係を示す因果構造マトリクス$M$がDAG(有向非循環グラフ)になるパターンを探索する必要がある .　そのため、因果構造マトリクス$M$がDAGでないときには損失を与え、DAGになるようにバックプロパゲーションで学習させる必要がある .

#### ・このマトリクスがDAGかどうかを判定する損失として、NO TEARS(Non-combinatorial Optimization via Trace Exponential and Augmented lagRangian for Structure learning)と呼ばれる手法が提案されている .　SAMでは損失関数にこのNO TEARSを利用する .

#### ただし、論文NO TEARSそのものはディープラーニングによる因果探索の提案ではなく、DAGとなる拘束条件の与え方を示す論文である .

#### ・NO TEARSによる損失計算の具体的な形は次の通りである .　因果構造マトリクス$M$がDAGの場合、以下の関係が成り立つ .

$$
\sum_{k=1}^d \frac{\mathrm{tr}\, M^k}{k!} = 0
$$


#### ここで、$d$は観測変数の種類数であり、マトリクスの行である .

#### ・因果構造マトリクス$M$がDAGではない場合、

$$
\sum_{k=1}^d \frac{\mathrm{tr}\, M^k}{k!} 
$$

#### の値が0にならず、正の値をとる .　そのため、この項を損失関数に加えることで、因果構造マトリクス$M$がDAGになるように$M$を学習させることができる .

#### ・NO TEARSの損失計算の実装は、SAMの著者らの実装をそのまま流用している .

### SAMの誤差関数

In [11]:
# ネットワークを示す因果構造マトリクスMがDAG（有向非循環グラフ）になるように加える損失

def notears_constr(adj_m, max_pow=None):
    """No Tears constraint for binary adjacency matrixes. 
    Args:
        adj_m (array-like): Adjacency matrix of the graph
        max_pow (int): maximum value to which the infinite sum is to be computed.
           defaults to the shape of the adjacency_matrix
    Returns:
        np.ndarray or torch.Tensor: Scalar value of the loss with the type
            depending on the input.
    参考：https://github.com/FenTechSolutions/CausalDiscoveryToolbox/blob/32200779ab9b63762be3a24a2147cff09ba2bb72/cdt/utils/loss.py#L215
    """
    m_exp = [adj_m]
    if max_pow is None:
        max_pow = adj_m.shape[1]
    while(m_exp[-1].sum() > 0 and len(m_exp) < max_pow):
        m_exp.append(m_exp[-1] @ adj_m/len(m_exp))

    return sum([i.diag().sum() for idx, i in enumerate(m_exp)])

## 識別器Dと生成器Gの損失関数

#### ・SAMの論文では通常のGAN(正確にはDCGAN)の損失計算以外にも、ベイジアンネットワークの因果探索でのスコアリング法で用いられるMDL(Minimum Description Length)に基づく損失関数の使用が提案されている .　しかし、MDLに基づくGANの損失関数の導出と解説が難しいため、一般的なDCGANでの損失を使用する .

#### ・まず識別器Dの損失関数です .　Binary cross entropy with Logistic functionと呼ばれる関数で損失を計算する .　PyTorchでは、Torch.nn.BCEWithLogisticLoss()として用意される .　式で記述すると、

$$
- \sum_{i=1}^{N} \left[ l_i \log y_i + (1 - l_i) \log (1 - y_i) \right]
$$

#### となる .　ここで、$l_i$は$i$番目のデータのラベル(データセットのデータなら1、生成器Gから生成していれば0)を示し、$y_i$は識別器の出力を示す .　$N$はミニバッチのデータ数である .

#### ・生成器Gは識別器Dを騙すようにしたいので、生成器Gの損失関数は識別器Dの損失関数にマイナスをかけた以下になる .

$$
\sum_{i=1}^{N} \left[ l_i \log y_i + (1 - l_i) \log (1 - y_i) \right]
$$

#### ここでは生成器Gだけを考えるため、$l_i$は0だけとなる .　そして$y_i$は生成されたデータを判定した結果なので、生成器Gの損失関数は、

$$
+ \sum_{i=1}^{N} \log \left(1 - D(G(z_i, x_i))\right)
$$


#### となる .　ここで$z_i$はデータ生成のためのノイズ、$x_i$は観測データである .　通常のDCGANでは生成時に観測データを使用しないが、SAMは観測データを1変数ずつ生成するため、生成器Gに観測データ$x_i$を与えている .

#### ただし、上記の損失の形式では生成器Gの学習が進みづらいことが判明しているため、生成器Gの損失関数には上記を基にした

$$
- \sum_{i=1}^{N} \log \left( D(G(z_i, x_i)) \right)
$$


#### を使用する .

## 生成器の複雑さの損失関数

#### ・因果構造マトリクス$M$と複雑さマトリクス$Z$は、マトリクスの要素が0から1に変わると、より複雑な生成過程を作ることができる .　できれば可能な限りシンプルでミニマムな要素数で生成過程を実現することが好ましい .

#### そこで、これらのマトリクスのうち、1である要素数の合計をそのまま損失関数として使用する .　以下の数式で表される .

$$
\frac{\lambda_s}{N} \sum_{i,j} m_{i,j} + \frac{\lambda_F}{N} \sum_{i,j} z_{i,j}
$$

#### ここで、${\lambda_s}$と${\lambda_F}$はこの損失の影響力を決める係数です .　$m_{i,j}$は因果構造マトリクス$M$の要素で0か1の値をとる .　$z_{i,j}$は複雑さマトリクス$Z$の要素で0か1の値をとる .

## SAMの学習を実装するコード

#### ・注意点の1つ目は、訓練epochでネットワークを学習させ、その後テストepochで因果構造マトリクス$M$と生成データの損失を求めている点である .　因果構造マトリクス$M$の要素は0か1の離散値だが、確率的に0か1に求まるので、実装コード内のテスト部分と損失関数計算部分では、0か1ではなく、1となる確率値を使用している .

#### そして、テストepoch数の平均をとることで、因果構造マトリクス$M$とデータ生成の損失を求めている .

#### ・もう1つの注意点は、NO TEARSによる損失は訓練epochを経るに従い、線形的に強く影響するように与えている .　初めからDAGを制約すると上手く生成器Gが学習しづらいため、DAGの制約は徐々に強くしていく .

In [12]:
from sklearn.preprocessing import scale
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm


def run_SAM(in_data, lr_gen, lr_disc, lambda1, lambda2, hlayers, nh, dnh, train_epochs, test_epochs, device):
    '''SAMの学習を実行する関数'''

    # ---------------------------------------------------
    # 入力データの前処理
    # ---------------------------------------------------
    list_nodes = list(in_data.columns)  # 入力データの列名のリスト
    data = scale(in_data[list_nodes].values)  # 入力データの正規化
    nb_var = len(list_nodes)  # 入力データの数 = d
    data = data.astype('float32')  # 入力データをfloat32型に
    data = torch.from_numpy(data).to(device)  # 入力データをPyTorchのテンソルに
    rows, cols = data.size()  # rowsはデータ数、colsは変数の数

    # ---------------------------------------------------
    # DataLoaderの作成（バッチサイズは全データ）
    # ---------------------------------------------------
    batch_size = rows  # 入力データ全てを使用したミニバッチ学習とする
    data_iterator = DataLoader(data, batch_size=batch_size,
                               shuffle=True, drop_last=True)
    # 注意：引数のdrop_lastはdataをbatch_sizeで取り出していったときに最後に余ったものは使用しない設定

    # ---------------------------------------------------
    # 【Generator】ネットワークの生成とパラメータの初期化
    # cols：入力変数の数、nhは中間ニューロンの数、hlayersは中間層の数
    # neuron_samplerは、Functional gatesの変数zを学習するネットワーク
    # graph_samplerは、Structual gatesの変数aを学習するネットワーク
    # ---------------------------------------------------
    sam = SAMGenerator((batch_size, cols), nh).to(device)  # 生成器G
    graph_sampler = MatrixSampler(nb_var, mask=None, gumbel=False).to(
        device)  # 因果構造マトリクスMを作るネットワーク
    neuron_sampler = MatrixSampler((nh, nb_var), mask=False, gumbel=True).to(
        device)  # 複雑さマトリクスZを作るネットワーク

    # 注意：MatrixSamplerはGumbel-Softmaxを使用し、0か1を出力させるニューラルネットワーク
    # SAMの著者らの実装モジュール、MatrixSamplerを使用
    # https://github.com/FenTechSolutions/CausalDiscoveryToolbox/blob/32200779ab9b63762be3a24a2147cff09ba2bb72/cdt/utils/torch.py#L212

    # 重みパラメータの初期化
    sam.reset_parameters()
    graph_sampler.weights.data.fill_(2)

    # ---------------------------------------------------
    # 【Discriminator】ネットワークの生成とパラメータの初期化
    # cols：入力変数の数、dnhは中間ニューロンの数、hlayersは中間層の数。
    # ---------------------------------------------------
    discriminator = SAMDiscriminator(cols, dnh, hlayers).to(device)
    discriminator.reset_parameters()  # 重みパラメータの初期化

    # ---------------------------------------------------
    # 最適化の設定
    # ---------------------------------------------------
    # 生成器

    g_optimizer = optim.Adam(sam.parameters(), lr=lr_gen)
    graph_optimizer = optim.Adam(graph_sampler.parameters(), lr=lr_gen)
    neuron_optimizer = optim.Adam(neuron_sampler.parameters(), lr=lr_gen)

    # 識別器
    d_optimizer = optim.Adam(discriminator.parameters(), lr=lr_disc)

    # 損失関数
    criterion = nn.BCEWithLogitsLoss()
    # nn.BCEWithLogitsLoss()は、binary cross entropy with Logistic function
    # https://pytorch.org/docs/stable/nn.html#bcewithlogitsloss

    # 損失関数のDAGに関する制約の設定パラメータ
    dagstart = 0.5
    dagpenalization_increase = 0.001*10

    # ---------------------------------------------------
    # forward計算、および損失関数の計算に使用する変数を用意
    # ---------------------------------------------------
    _true = torch.ones(1).to(device)
    _false = torch.zeros(1).to(device)

    noise = torch.randn(batch_size, nb_var).to(device)  # 生成器Gで使用する生成ノイズ
    noise_row = torch.ones(1, nb_var).to(device)

    output = torch.zeros(nb_var, nb_var).to(device)  # 求まった隣接行列
    output_loss = torch.zeros(1, 1).to(device)

    # ---------------------------------------------------
    # forwardの計算で、ネットワークを学習させる
    # ---------------------------------------------------
    pbar = tqdm(range(train_epochs + test_epochs))  # 進捗（progressive bar）の表示

    for epoch in pbar:
        for i_batch, batch in enumerate(data_iterator):

            # 最適化を初期化
            g_optimizer.zero_grad()
            graph_optimizer.zero_grad()
            neuron_optimizer.zero_grad()
            d_optimizer.zero_grad()

            # 因果構造マトリクスM（drawn_graph）と複雑さマトリクスZ（drawn_neurons）をMatrixSamplerから取得
            drawn_graph = graph_sampler()
            drawn_neurons = neuron_sampler()
            # (drawn_graph)のサイズは、torch.Size([nb_var, nb_var])。 出力値は0か1
            # (drawn_neurons)のサイズは、torch.Size([nh, nb_var])。 出力値は0か1

            # ノイズをリセットし、生成器Gで疑似データを生成
            noise.normal_()
            generated_variables = sam(data=batch, noise=noise,
                                      adj_matrix=torch.cat(
                                          [drawn_graph, noise_row], 0),
                                      drawn_neurons=drawn_neurons)

            # 識別器Dで判定
            # 観測変数のリスト[]で、各torch.Size([data数, 1])が求まる
            disc_vars_d = discriminator(generated_variables.detach(), batch)
            # 観測変数のリスト[] で、各torch.Size([data数, 1])が求まる
            disc_vars_g = discriminator(generated_variables, batch)
            true_vars_disc = discriminator(batch)  # torch.Size([data数, 1])が求まる

            # 損失関数の計算（DCGAN）
            disc_loss = sum([criterion(gen, _false.expand_as(gen)) for gen in disc_vars_d]) / nb_var \
                + criterion(true_vars_disc, _true.expand_as(true_vars_disc))

            gen_loss = sum([criterion(gen,
                                      _true.expand_as(gen))
                            for gen in disc_vars_g])
            
            # 損失の計算（SAM論文のオリジナルのfgan）
            #disc_loss = sum([torch.mean(torch.exp(gen - 1)) for gen in disc_vars_d]) / nb_var - torch.mean(true_vars_disc)
            #gen_loss = -sum([torch.mean(torch.exp(gen - 1)) for gen in disc_vars_g])

            # 識別器Dのバックプロパゲーションとパラメータの更新
            if epoch < train_epochs:
                disc_loss.backward()
                d_optimizer.step()

            # 生成器のGの損失の計算の残り（マトリクスの複雑さとDAGのNO TEAR）
            struc_loss = lambda1 / batch_size*drawn_graph.sum()     # Mのloss
            func_loss = lambda2 / batch_size*drawn_neurons.sum()   # Aのloss

            regul_loss = struc_loss + func_loss

            if epoch <= train_epochs * dagstart:
                # epochが基準前のときは、DAGになるようにMへのNO TEARSの制限はかけない
                loss = gen_loss + regul_loss

            else:
                # epochが基準後のときは、DAGになるようにNO TEARSの制限をかける
                filters = graph_sampler.get_proba()  # マトリクスMの要素を取得（ただし、0,1ではなく、1の確率）
                dag_constraint = notears_constr(filters*filters)  # NO TERARの計算

                # 徐々に線形にDAGの正則を強くする
                loss = gen_loss + regul_loss + \
                    ((epoch - train_epochs * dagstart) *
                     dagpenalization_increase) * dag_constraint

            if epoch >= train_epochs:
                # testのepochの場合、結果を取得
                output.add_(filters.data)
                output_loss.add_(gen_loss.data)
            else:
                # trainのepochの場合、生成器Gのバックプロパゲーションと更新
                # retain_graph=Trueにすることで、以降3つのstep()が実行できる
                loss.backward(retain_graph=True)
                g_optimizer.step()
                graph_optimizer.step()
                neuron_optimizer.step()

                # 進捗の表示
            if epoch % 50 == 0:
                pbar.set_postfix(gen=gen_loss.item()/cols,
                                 disc=disc_loss.item(),
                                 regul_loss=regul_loss.item(),
                                 tot=loss.item())

    return output.cpu().numpy()/test_epochs, output_loss.cpu().numpy()/test_epochs/cols  # Mと損失を出力

## 8-5 Google ColaboratoryでGPUを使用した因果探索の実行

## Google ColaboratoryでGPUを使用する方法

#### ・Google ColaboratoryでGPUを24時間以内に最長で12時間使用できる .

#### ・GPU利用の手順を解説する .　ノートブックを開き、上部メニューの「ランタイム」を選択し、展開されたメニューから「ランタイムのタイプを変更」をクリックする(図8.5.1) .

#### ・ノートブックの設定が開くので、「ハードウェアアクセラレータ」を"GPU"に設定し、右下の「保存」をクリックする(図8.5.2) .

![alt text](pict8.png)

![alt text](pict9.png)

#### ・以下のコードを実行して、PyTorchからGPUが使用できるかを確認する .

### GPUの使用可能を確認

In [ ]:
# GPUの使用確認：True or False
torch.cuda.is_available()
# 出力がTrueであれば、GPU使用の設定が完了.

False

## SAMの学習を実施

#### ・SAMはGANを使った確率的な因果探索手法なので、結果は毎回変化する .　そこで、SAMの著者らは8回以上実行し、求まった結果の平均を使用することを推奨している .

#### しかし、8回以上の実行は時間がかかるので、今回は5回の因果探索結果の平均を求めるように実行する .

#### ・実装は以下の通りである .　なお、GPUで使われる乱数生成のseedを固定していないので実行結果は毎回微妙に異なる(PyTorchでGPU部分もseedを固定できるが、実行速度が落ちるので、今回は固定しない) .

In [ ]:
# numpyの出力を小数点2桁に
np.set_printoptions(precision=2, floatmode='fixed', suppress=True)

# 因果探索の結果を格納するリスト
m_list = []
loss_list = []

for i in range(5):
    m, loss = run_SAM(in_data=df, lr_gen=0.01*0.5,
                      lr_disc=0.01*0.5*2,
                      #lambda1=0.01, lambda2=1e-05,
                      lambda1=5.0*20, lambda2=0.005*20,
                      hlayers=2,
                      nh=200, dnh=200,
                      train_epochs=10000,
                      test_epochs=1000,
                      device='cuda:0')

    print(loss)
    print(m)

    m_list.append(m)
    loss_list.append(loss)

# ネットワーク構造（5回の平均）
print(sum(m_list) / len(m_list))

# mはこうなって欲しい
#    x Z Y Y2 Y3 Y4
# x  0 1 1 0 0 0
# Z  0 0 1 0 0 0
# Y  0 0 0 0 1 0
# Y2 0 0 0 0 1 0
# Y3 0 0 0 0 0 1
# Y4 0 0 0 0 0 0

#### ・下の図がSAMを実行した結果の様子である .　テストepochでの生成データの損失平均と、最終的に求まった因果構造マトリクス$M$のテストepochでの平均が、5試行分出力されている .

![alt text](pict10.png)

#### ・正解のネットワークは、変数$(x,Z,Y,Y2,Y3,Y4)$に対して、

$$
M_{ans} =
\begin{pmatrix}
0 & 1 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$$

#### であったが、SAMで因果探索を実行した結果、以下のように求まった .

$$
M_{inference} =
\begin{pmatrix}
0.00 & 0.81 & 0.62 & 0.01 & 0.14 & 0.03 \\
0.11 & 0.00 & 0.63 & 0.00 & 0.20 & 0.26 \\
0.16 & 0.42 & 0.00 & 1.00 & 0.85 & 0.34 \\
0.20 & 0.00 & 0.00 & 0.00 & 0.09 & 0.01 \\
0.05 & 0.00 & 0.03 & 0.99 & 0.00 & 0.84 \\
0.06 & 0.01 & 0.08 & 0.13 & 0.25 & 0.00 \\
\end{pmatrix}
$$

#### ・ここで、適当に閾値を0.6と設定し、0.6以上であれば1、それ以下であれば0とすると、

$$
M_{inference} =
\begin{pmatrix}
0 & 1 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$$

#### となる .　正解の因果ダイアグラムと、SAMによる因果探索の結果を図で示すと以下のようになる .　探索した結果、求まったネットワークは完全に正確な結果ではないが、大まかな骨子はうまく推定できているように感じる .　一方で因果の矢印が逆になっている部分も見られる .

#### SAMのハイパーパラメータをチューニングしたりすると、もう少し正しい結果が得られるかもしれない .　SAMはハイパーパラメータが多く、その設定が難しいところが課題であると著者は感じている .

![alt text](pict11.png)